# Web scrapper

In [ ]:
!apt update
!apt install g++ openjdk-8-jdk openjdk-8-jre 
!pip install pandas numpy tqdm jnius gevent JPype1-py3 konlpy

In [ ]:
import utils

utils.database.generate_db_directory()


In [ ]:
start_page_num = 6815073
end_page_num = 6989888

utils.parsers.run_web_crawler(start_page_num,end_page_num)

# tagging

In [ ]:
import utils
wordranker = utils.parsers.WordRanker(use_cache=False)
wordranker.run()
wordranker.word_cloud()

# PPT

In [ ]:
import utils

yotube_downloader = utils.youtube.YoutubeDownloader()
yotube_downloader.run()

In [ ]:
import utils

utils.pptx_utils.update_slide(0, 'sum_ai.csv')
utils.pptx_utils.update_slide(2, 'sum_ai.csv')
utils.pptx_utils.update_slide(3, 'sum_noholo.csv')
utils.pptx_utils.update_slide(4, 'youtube.csv')
utils.pptx_utils.save_ppt()

In [ ]:
import pandas as pd
import utils
import os

def inc_percentage(a, b):
    return 100 * (b - a) / a

prev_post_file = os.path.join(utils.parsers.PREV_DIR, utils.parsers.POST_FILENAME)

prev_post_content = pd.read_json(prev_post_file)
post_content = pd.read_json(utils.parsers.POST_FILE)
cur_post_num = post_content['번호'].count()
old_post_num = prev_post_content['번호'].count()
percentage = inc_percentage(old_post_num, cur_post_num)
print(f"total number of post: {cur_post_num}")
print(f"inc percentage number of post: {round(percentage, 1)}")
print(post_content.sort_values(by=['추천 수'], ascending=False).head(n=1))
print("\n")
print(post_content.sort_values(by=['조회 수'], ascending=False).head(n=1))
print("\n")
print(post_content.sort_values(by=['달린 댓글 수'], ascending=False).head(n=1))
print("\n")
print(post_content["개념글 수"].value_counts())

# Dccon Rank

In [ ]:
import utils
wordranker = utils.parsers.wordRanker()
wordranker.dccon_rank()
wordranker.dccon_download()

In [ ]:
import utils
youtuberanker = utils.parsers.youtubeCounter()
youtuberanker.run()